In [59]:
import torch
import torch.nn as nn
import torch.optim as optim
from tools import *
from config import Params
from tqdm import tqdm

In [72]:
args = Params(use_cuda=True, debug=False, data=r'./processed_signal/HKU956/956_772_12s_step_4s.pkl', batch_size=128,
            spliter=r'./processed_signal/HKU956/956_772_12s_step_4s_spliter5.pkl',)

spliter = load_model(args.spliter)
data = pd.read_pickle(args.data)

for i, k in enumerate(spliter[args.valid]):
    args.k = i
    print("\n" + "=======" * 6 + '[Fold {}]'.format(i), "=======" * 6)
    train_index = k['train_index']
    test_index = k['test_index']
    break

dataprepare = DataPrepare(args,
                                  target=args.target,
                                  data=data,
                                  train_index=train_index,
                                  test_index=test_index,
                                  device=args.device,
                                  batch_size=args.batch_size)
train_dataloader, test_dataloader = dataprepare.get_data()


==========================================[Fold 0] ==========================================
Target distribution:
1    552
0    404
dtype: int64
(764, 4, 768) (764, 1) (192, 4, 768) (192, 1)
Train target distribution
1    441
0    323
dtype: int64
Test target distribution
1    111
0     81
dtype: int64


In [73]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.device = device
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        h0 = torch.zeros(1, x.size(0), self.hidden_dim).requires_grad_().to(self.device)
        c0 = torch.zeros(1, x.size(0), self.hidden_dim).requires_grad_().to(self.device)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])
        return out

In [74]:
# define the training function
def train(model, optimizer, criterion, train_loader, test_loader, n_epochs):
    for epoch in range(n_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in tqdm(enumerate(train_loader)):
            labels = labels.flatten()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print('Epoch [%d/%d], Loss: %.4f' % (epoch+1, n_epochs, running_loss / len(train_loader)))

        test(model, criterion, test_loader)

def test(model, criterion, test_loader):
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for inputs, labels in test_loader:
            labels = labels.flatten()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()
        accuracy = 100.0 * total_correct / total_samples
        print('Test Accuracy: %.2f%% (%d/%d)' % (accuracy, total_correct, total_samples))

In [75]:
input_dim = 4
hidden_dim = 32
output_dim = 2
learning_rate = 0.01
n_epochs = 100

# create the model
model = LSTMClassifier(input_dim, hidden_dim, output_dim, device=args.device)
model = model.to(args.device)

# define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# create the dataset and dataloader
train_data = torch.randn(100, 10, input_dim)
train_labels = torch.randint(0, output_dim, (100,))
train_dataset = torch.utils.data.TensorDataset(train_data, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

# train the model
# train(model, optimizer, criterion, train_loader, n_epochs)
train(model, optimizer, criterion, train_dataloader, test_dataloader, n_epochs)


6it [00:01,  3.76it/s]


Epoch [1/100], Loss: 0.6820
Test Accuracy: 57.81% (111/192)


6it [00:00, 21.91it/s]


Epoch [2/100], Loss: 0.6820
Test Accuracy: 57.81% (111/192)


6it [00:00, 19.68it/s]


...

Epoch [98/100], Loss: 0.6830
Test Accuracy: 55.73% (107/192)


6it [00:00, 21.90it/s]


Epoch [99/100], Loss: 0.6831
Test Accuracy: 57.81% (111/192)


6it [00:00, 21.51it/s]

Epoch [100/100], Loss: 0.6821
Test Accuracy: 57.81% (111/192)
